# Лабораторная работа 2
## Задание:
1. Провести классификацию найденного датасета, методом машины опорных векторов. В формате Markdown писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

### Подключение библиотек

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC

### Загрузка данных и подготовка датасета

In [2]:
DataSet = pd.read_csv('property_assessment.csv')

label_encoder = LabelEncoder()
DataSet['COND_ENC'] = label_encoder.fit_transform(DataSet['OVERALL_COND'])
DataSet['CITY_ENC'] = label_encoder.fit_transform(DataSet['CITY'])
DataSet['BLDG_TYPE_ENC'] = label_encoder.fit_transform(DataSet['BLDG_TYPE'])
DataSet[['TOTAL_VALUE', 'LAND_VALUE', 'BLDG_VALUE']] = DataSet[['TOTAL_VALUE', 'LAND_VALUE', 'BLDG_VALUE']].applymap(lambda x: x.replace(',', '') if isinstance(x, str) else x).apply(pd.to_numeric, errors='coerce')
non_numeric_columns = DataSet.select_dtypes(exclude=['number']).columns.tolist()

DataSet = DataSet.drop(columns=non_numeric_columns)
DataSet.dropna(inplace=True)
print(DataSet)

              PID         CM_ID      GIS_ID  ST_NUM  ZIP_CODE  BLDG_SEQ  \
0       100018006  1.000180e+08   100018000   239.0    2128.0         1   
1       100018008  1.000180e+08   100018000   239.0    2128.0         1   
2       100019006  1.000190e+08   100019000   241.0    2128.0         1   
3       100024002  1.000240e+08   100024000   249.0    2128.0         1   
4       100041006  1.000410e+08   100041000   224.0    2128.0         1   
...           ...           ...         ...     ...       ...       ...   
10316  2205550698  2.205550e+09  2205550001   116.0    2135.0         1   
10317  2205550706  2.205550e+09  2205550001   112.0    2135.0         1   
10318  2205550714  2.205550e+09  2205550001   108.0    2135.0         1   
10319  2205550726  2.205550e+09  2205550001   100.0    2135.0         1   
10320  2205550730  2.205550e+09  2205550001   100.0    2135.0         1   

       NUM_BLDGS  LUC  RES_FLOOR  GROSS_AREA  ...  BED_RMS  FULL_BTH  HLF_BTH  \
0              1  

### Нормализация данных

In [3]:
X = DataSet[['BLDG_TYPE_ENC', 'LIVING_AREA', 'TOTAL_VALUE', 'CITY_ENC']]
y = DataSet['COND_ENC']  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Обучение и оценка качества на линейном ядре


In [4]:
param_grid = {'kernel': ['linear', 'rbf', 'sigmoid', 'poly'],
              'C': [.01, 0.1, 1, 10],
              'degree': [1, 2, 3, 4, 5]}  

svm_grid = GridSearchCV(SVC(), param_grid, cv=3, scoring='accuracy', n_jobs=-1)

svm_grid.fit(X_test, y_test)

best_params = svm_grid.best_params_
best_kernel = best_params['kernel']
best_C = best_params['C']
best_degree = best_params['degree']

print(f"Лучшие параметры: Ядро = {best_kernel}, C = {best_C}, Степень = {best_degree}")

Лучшие параметры: Ядро = poly, C = 1, Степень = 3


### Повторное обучение на лучших параметрах

In [5]:
svm_model = SVC(kernel='poly', degree=3, C=1) 

# Обучение модели
svm_model.fit(X_train, y_train)

# Предсказание на тестовом наборе
y_pred = svm_model.predict(X_test)

# Оценка модели
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, zero_division=1)

print(f'Точность модели: {accuracy}')
print(report)

# Кросс-валидация
stratified_kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
cross_val_scores = cross_val_score(svm_model, X_train, y_train, cv=stratified_kfold)

print(f'Средняя точность перекрестной проверки: {cross_val_scores.mean()}')

Точность модели: 0.7398885928796318
              precision    recall  f1-score   support

           0       0.74      1.00      0.85      3050
           1       0.75      0.05      0.09        62
           2       1.00      0.05      0.10        60
           3       1.00      0.00      0.00         1
           4       1.00      0.00      0.00       915
           5       1.00      0.00      0.00        41

    accuracy                           0.74      4129
   macro avg       0.91      0.18      0.17      4129
weighted avg       0.80      0.74      0.63      4129

Средняя точность перекрестной проверки: 0.7425710594315245
